# 1. Redução das das amostras 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.colors import ListedColormap
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.multiclass import OneVsRestClassifier


from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier,MLPRegressor

import sklearn
import random

In [125]:
def read_instance(filepath):
    data = pd.read_csv(filepath)
    return(data)

In [131]:
def clean_data(data):
    data = read_instance(data)
    #retirando a coluna de nomes, local de embarque, tarifa do passageiro, 
    #numero do ticket 
    data = data.drop(columns=['Name'])
    data = data.drop(columns=['Ticket'])
    data = data.drop(columns=['Fare'])
    data = data.drop(columns=['Embarked'])
    # escolher se tirar cabine ou não pois a cabine infere no setor do barco
    data = data.drop(columns=['Cabin'])
    data = data.dropna()
    #trocando valores de male e female por 1 e 0
    data.loc[data.Sex=='male','Sex'] = 1
    data.loc[data.Sex=='female','Sex'] = 0
    return data

In [134]:
data = clean_data('train.csv')

In [135]:
display(data)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch
0,1,0,3,1,22.0,1,0
1,2,1,1,0,38.0,1,0
2,3,1,3,0,26.0,0,0
3,4,1,1,0,35.0,1,0
4,5,0,3,1,35.0,0,0
...,...,...,...,...,...,...,...
885,886,0,3,0,39.0,0,5
886,887,0,2,1,27.0,0,0
887,888,1,1,0,19.0,0,0
889,890,1,1,1,26.0,0,0


In [136]:
data_test = clean_data('test.csv')

In [137]:
display(data_test)

,PassengerId,Pclass,Sex,Age,SibSp,Parch
0,892,3,1,34.5,0,0
1,893,3,0,47.0,1,0
2,894,2,1,62.0,0,0
3,895,3,1,27.0,0,0
4,896,3,0,22.0,1,1
...,...,...,...,...,...,...
409,1301,3,0,3.0,1,1
411,1303,1,0,37.0,1,0
412,1304,3,0,28.0,0,0
414,1306,1,0,39.0,0,0


# Treino

In [10]:
x,y = data.drop(columns=['Survived']), data['Survived']

In [11]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3)

In [12]:
y_train

408    0
858    1
175    0
704    0
63     0
      ..
81     1
684    0
606    0
279    1
816    0
Name: Survived, Length: 499, dtype: int64

In [118]:
model = MLPClassifier(activation='logistic', max_iter=10000,solver ='adam',hidden_layer_sizes = (100))
model.fit(x_train, y_train)

              precision    recall  f1-score   support

           0       0.76      0.88      0.82       121
           1       0.81      0.64      0.71        94

    accuracy                           0.78       215
   macro avg       0.78      0.76      0.77       215
weighted avg       0.78      0.78      0.77       215



In [139]:
y_pred = model.predict(data_test)
#print(classification_report(y_test,y_pred))#possivel overfit

In [140]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0,